In [1]:
%cd ../..

/home/kiwi/Desktop/Example-RAG


In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

from src.example_rag.chatbot.messenger import Messenger
from src.example_rag.chatbot.model import MiniLlama
from src.example_rag.chatbot.retriever import get_retriever
from src.example_rag.settings import COLLECTION_NAME, GenerationSettings

gen_settings = GenerationSettings()

chat_model = MiniLlama(
    max_new_tokens=gen_settings.max_new_tokens,
    temperature=gen_settings.temperature,
    top_p=gen_settings.top_p,
)
retriever = get_retriever(COLLECTION_NAME, gen_settings.similiarity_top_k, host='localhost')
messenger =  Messenger(chat_model, retriever)

In [ ]:
questions = [
    "Who was the first love of Liudas Vasaris?",
]

ground_truths = [
    "His first love was Lucie",
]

answers = []

contexts = []

for question in questions:
    answer, nodes = messenger.chat(question, return_nodes=True)
    context = [node.text for node in nodes]
    messenger.reset_context()
    answers.append(answer)
    contexts.append(context)


data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

result = result.to_pandas()
result.head()